<a href="https://colab.research.google.com/github/harsh-agar/frustum_pointnets/blob/master/pointnet_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import numpy as np
  
a = np.arange(5) 
  
# a is printed. 
print("a is:") 
print(a) 
  
# the array is saved in the file geekfile.npy  
np.save('/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/models/train_feats', a) 
np.save('/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/models/val_feats', a) 
np.save('/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/models/train_labels', a) 
np.save('/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/models/val_labels', a)
np.save('/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/train_feats', a) 
np.save('/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/val_feats', a) 
np.save('/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/train_labels', a) 
np.save('/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/val_labels', a)
np.save('train_feats', a) 
np.save('val_feats', a) 
np.save('train_labels', a) 
np.save('val_labels', a)

a is:
[0 1 2 3 4]


In [4]:
from __future__ import print_function

import os
import sys
import argparse
import importlib
import numpy as np
import tensorflow as tf
from datetime import datetime
print ("1")
ROOT_DIR = '/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/'
BASE_DIR = '/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/'
sys.path.append('/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/')
sys.path.append('/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/models/')
sys.path.append('/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/train/')
import provider_1 as provider
print ("2")
from model_util import MODEL, LOG_DIR, NUM_POINT
from model_util import BATCH_SIZE, NUM_POINT, MAX_EPOCH
from model_util import LEARNING_RATE, GPU_INDEX, MOMENTUM
from model_util import OPTIMIZER, DECAY_STEP, DECAY_RATE
from model_util import NUM_FEATURES, NUM_CLASSES, RESTORE_MODEL_PATH


# Set training configurations
EPOCH_CNT = 0
print ("3")

MODEL_FILE = os.path.join(ROOT_DIR, 'models', MODEL+'.py')
MODEL = importlib.import_module(MODEL) # import network module
LOG_DIR = LOG_DIR
if not os.path.exists(LOG_DIR): os.mkdir(LOG_DIR)
os.system('cp %s %s' % (MODEL_FILE, LOG_DIR)) # bkp of model def
os.system('cp %s %s' % (os.path.join(BASE_DIR, 'train.py'), LOG_DIR))
LOG_FOUT = open(os.path.join(LOG_DIR, 'log_train.txt'), 'w')
# LOG_FOUT.write(str(FLAGS)+'\n')

BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
BN_DECAY_DECAY_STEP = float(DECAY_STEP)
BN_DECAY_CLIP = 0.99

# # Load Frustum Datasets. Use default data paths.
# TRAIN_DATASET_FEATS = np.load('train_feats.npy')
# TRAIN_DATASET_LABELS = np.load('train_labels.npy')
# TEST_DATASET_FEATS = np.load('train_feats.npy')
# TEST_DATASET_LABELS = np.load('train_labels.npy')

TRAIN_FILES = provider.getDataFiles( \
    os.path.join(BASE_DIR, '/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/data/modelnet40_ply_hdf5_2048/train_files.txt'))
TEST_FILES = provider.getDataFiles(\
    os.path.join(BASE_DIR, '/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/data/modelnet40_ply_hdf5_2048/test_files.txt'))


1
2
3


In [0]:
def log_string(out_str):
    LOG_FOUT.write(out_str+'\n')
    LOG_FOUT.flush()
    print(out_str)

def get_learning_rate(batch):
    learning_rate = tf.train.exponential_decay(
                        LEARNING_RATE,  # Base learning rate.
                        batch * BATCH_SIZE,  # Current index into the dataset.
                        DECAY_STEP,          # Decay step.
                        DECAY_RATE,          # Decay rate.
                        staircase=True)
    learing_rate = tf.maximum(learning_rate, 0.00001) # CLIP THE LEARNING RATE!
    return learning_rate        

def get_bn_decay(batch):
    bn_momentum = tf.train.exponential_decay(
                      BN_INIT_DECAY,
                      batch*BATCH_SIZE,
                      BN_DECAY_DECAY_STEP,
                      BN_DECAY_DECAY_RATE,
                      staircase=True)
    bn_decay = tf.minimum(BN_DECAY_CLIP, 1 - bn_momentum)
    return bn_decay

In [0]:
def train_one_epoch(sess, ops, train_writer):
    ''' Training for one epoch on the frustum dataset.
    ops is dict mapping from string to tf ops
    '''
    is_training = True
    log_string(str(datetime.now()))
    
    # Shuffle train samples
    train_file_idxs = np.arange(0, len(TRAIN_FILES))
    np.random.shuffle(train_file_idxs)
#     num_batches = int(len(TRAIN_DATASET_LABELS)/BATCH_SIZE)

    # To collect statistics
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    label_correct_cnt = 0


    for fn in range(len(TRAIN_FILES)):
        log_string('----' + str(fn) + '-----')
        current_data, current_label = provider.loadDataFile(TRAIN_FILES[train_file_idxs[fn]])
        current_data = current_data[:,0:NUM_POINT,:]
        current_data, current_label, _ = provider.shuffle_data(current_data, np.squeeze(current_label))            
        current_label = np.squeeze(current_label)
        
        file_size = current_data.shape[0]
        num_batches = file_size // BATCH_SIZE
        
        total_correct = 0
        total_seen = 0
        loss_sum = 0
       
        for batch_idx in range(num_batches):
            start_idx = batch_idx * BATCH_SIZE
            end_idx = (batch_idx+1) * BATCH_SIZE
            
            # Augment batched point clouds by rotation and jittering
            rotated_data = provider.rotate_point_cloud(current_data[start_idx:end_idx, :, :])
            jittered_data = provider.jitter_point_cloud(rotated_data)

            feed_dict = {ops['pointclouds_pl']: jittered_data,
                         ops['class_pl']: current_label[start_idx:end_idx],
                         ops['is_training_pl']: is_training}

            summary, step, _, loss_val, label_logits_val = \
                sess.run([ops['merged'], ops['step'], ops['train_op'], ops['loss'],
                    ops['class_logits']], feed_dict=feed_dict)

            train_writer.add_summary(summary, step)

            label_preds_val = np.argmax(label_logits_val, 1)
            correct = np.sum(label_preds_val == current_label[start_idx:end_idx])
            total_correct += correct
            total_seen += (BATCH_SIZE)
            loss_sum += loss_val
 
        log_string('mean loss: %f' % (loss_sum / float(num_batches)))
        log_string('accuracy: %f' % (total_correct / float(total_seen)))


In [0]:
def eval_one_epoch(sess, ops, test_writer):
    ''' Simple evaluation for one epoch on the frustum dataset.
    ops is dict mapping from string to tf ops """
    '''
    global EPOCH_CNT
    is_training = False
    log_string(str(datetime.now()))
    log_string('---- EPOCH %03d EVALUATION ----'%(EPOCH_CNT))
    # test_idxs = np.arange(0, len(TEST_DATASET_LABELS))
    # num_batches = int(len(TEST_DATASET_LABELS)/BATCH_SIZE)

    # To collect statistics
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    total_seen_class = [0 for _ in range(NUM_CLASSES)]
    total_correct_class = [0 for _ in range(NUM_CLASSES)]
       
    print(len(TEST_FILES))
    for fn in range(len(TEST_FILES)-1):
        log_string('----' + str(fn) + '-----')
        current_data, current_label = provider.loadDataFile(TEST_FILES[fn])
        current_data = current_data[:,0:NUM_POINT,:]
        current_label = np.squeeze(current_label)
        
        file_size = current_data.shape[0]
        num_batches = file_size // BATCH_SIZE
        
        for batch_idx in range(num_batches):
            start_idx = batch_idx * BATCH_SIZE
            end_idx = (batch_idx+1) * BATCH_SIZE
        print ((current_data[start_idx:end_idx, :, :]).shape)
        feed_dict = {ops['pointclouds_pl']: current_data[start_idx:end_idx, :, :],
                     ops['class_pl']: current_label[start_idx:end_idx],
                     ops['is_training_pl']: is_training}

        summary, step, loss_val, label_logits_val = \
            sess.run([ops['merged'], ops['step'], ops['loss'],
                ops['class_logits']], feed_dict=feed_dict)
        test_writer.add_summary(summary, step)

        label_preds_val = np.argmax(label_logits_val, 1)
        correct = np.sum(label_preds_val == current_label[start_idx:end_idx])
        total_correct += correct
        total_seen += BATCH_SIZE
        loss_sum += loss_val
        for i in range(start_idx, end_idx):
            l = current_label[i]
            total_seen_class[l] += 1
            total_correct_class[l] += (label_preds_val[i-start_idx] == l)


    log_string('eval mean loss: %f' % (loss_sum / float(num_batches)))
    log_string('eval classification accuracy: %f'% \
        (total_correct / float(total_seen)))
         
    EPOCH_CNT += 1

In [0]:
os.chdir('/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/')

In [0]:
NUM_FEATURES=3
NUM_CLASSES=40

In [0]:
with tf.Graph().as_default():
    with tf.device('/gpu:'+str(GPU_INDEX)):
        # pointclouds_pl, class_pl = \
        #     MODEL.placeholder_inputs(BATCH_SIZE, NUM_POINT)
        pointclouds_pl = tf.placeholder(tf.float32,
            shape=(BATCH_SIZE, NUM_POINT, NUM_FEATURES))
        class_pl = tf.placeholder(tf.uint8, shape=(BATCH_SIZE,))
        is_training_pl = tf.placeholder(tf.bool, shape=())

        # Note the global_step=batch parameter to minimize. 
        # That tells the optimizer to increment the 'batch' parameter
        # for you every time it trains.
        batch = tf.get_variable('batch', [],
            initializer=tf.constant_initializer(0), trainable=False)
        bn_decay = get_bn_decay(batch)
        tf.summary.scalar('bn_decay', bn_decay)

        # Get model and losses 
        end_points = MODEL.get_model(pointclouds_pl,
            is_training_pl, bn_decay=bn_decay)
        print(class_pl)
        print("+++++++++++++")
        loss = MODEL.get_loss(class_pl, end_points)
        tf.summary.scalar('loss', loss)

        losses = tf.get_collection('losses')
        total_loss = tf.add_n(losses, name='total_loss')
        tf.summary.scalar('total_loss', total_loss)

        correct = tf.equal(tf.argmax(end_points['class_logits'], 1),
            tf.to_int64(class_pl))
        accuracy = tf.reduce_sum(tf.cast(correct, tf.float32)) / \
            float(BATCH_SIZE)
        tf.summary.scalar('classification accuracy', accuracy)

        # Get training operator
        learning_rate = get_learning_rate(batch)
        tf.summary.scalar('learning_rate', learning_rate)
        if OPTIMIZER == 'momentum':
            optimizer = tf.train.MomentumOptimizer(learning_rate,
                momentum=MOMENTUM)
        elif OPTIMIZER == 'adam':
            optimizer = tf.train.AdamOptimizer(learning_rate)
        train_op = optimizer.minimize(loss, global_step=batch)

        # Add ops to save and restore all the variables.
        saver = tf.train.Saver()

    # Create a session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.allow_soft_placement = True
    config.log_device_placement = False
    sess = tf.Session(config=config)

    # Add summary writers
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'train'), sess.graph)
    test_writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'test'), sess.graph)

    # Init variables
    if RESTORE_MODEL_PATH is None:
        init = tf.global_variables_initializer()
        sess.run(init)
    else:
        saver.restore(sess, FLAGS.RESTORE_MODEL_PATH)

    ops = {'pointclouds_pl': pointclouds_pl,
           'class_pl': class_pl,
           'is_training_pl': is_training_pl,
           'class_logits': end_points['class_logits'],
           'loss': loss,
           'train_op': train_op,
           'merged': merged,
           'step': batch,
           'end_points': end_points}

    for epoch in range(MAX_EPOCH):
        log_string('**** EPOCH %03d ****' % (epoch))
        sys.stdout.flush()
             
        train_one_epoch(sess, ops, train_writer)
        eval_one_epoch(sess, ops, test_writer)

        # Save the variables to disk.
        if epoch % 10 == 0:
            save_path = saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"))
            log_string("Model saved in file: %s" % save_path)


Tensor("Placeholder_1:0", shape=(512,), dtype=uint8, device=/device:GPU:0)
+++++++++++++
Tensor("Placeholder_1:0", shape=(512,), dtype=uint8, device=/device:GPU:0)
**** EPOCH 000 ****
2019-08-29 10:13:59.037878
----0-----
mean loss: 0.161330
accuracy: 0.086589
----1-----
mean loss: 0.131606
accuracy: 0.118164
----2-----
mean loss: 0.129182
accuracy: 0.123047
----3-----
mean loss: 0.136035
accuracy: 0.128906
----4-----
mean loss: 0.126696
accuracy: 0.125977
2019-08-29 10:14:06.384798
---- EPOCH 000 EVALUATION ----
2
----0-----
(512, 64, 3)
eval mean loss: 0.018553
eval classification accuracy: 0.083984
Model saved in file: log/model.ckpt
**** EPOCH 001 ****
2019-08-29 10:14:07.497501
----0-----
mean loss: 0.122112
accuracy: 0.128906
----1-----
mean loss: 0.122275
accuracy: 0.129395
----2-----
mean loss: 0.123229
accuracy: 0.143229
----3-----
mean loss: 0.124077
accuracy: 0.142090
----4-----
mean loss: 0.120286
accuracy: 0.133789
2019-08-29 10:14:14.422070
---- EPOCH 001 EVALUATION ----


In [0]:
!cp '/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/models/tf_util.py' '/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/tf_util.py'


!cp '/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/models/model_util.py' '/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/model_util.py'


In [0]:
!cp '/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/models/frustum_pointnets_v1.py' '/content/gdrive/My Drive/work/frustum_pointnet_radar/frustum-pointnets-master/frustum_pointnets_v1.py'